## Use Decision Tree for forcast IMOEX price
### Vol 1
#### 10.06.19

In [1]:
# connecting nessesary paceges
from pandas import read_csv as read
from pandas import DataFrame as DF
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from IPython.display import SVG
from IPython.display import display
from graphviz import Source
from IPython.display import HTML
style = "<style>svg{width:80% !important;height:80% !important;}</style>"
HTML(style)

In [2]:
# loading training and test data
train_data = read('https://raw.githubusercontent.com/Aurilino/Python/master/Decision%20Tree%20IMOEX/export_Index_min_07.06.txt')
test_data = read('https://raw.githubusercontent.com/Aurilino/Python/master/Decision%20Tree%20IMOEX/export_Index_min_10.06.txt')
train_data = train_data.drop('Time', axis=1)
test_data = test_data.drop('Time', axis=1)

In [3]:
# Get MOEX daly return with training data
MOEX_1 = DF(train_data.MOEX)
train_data = train_data.drop('MOEX', axis=1)
train_data = pd.merge(train_data, MOEX_1, left_index=True, right_index=True)
train_data.columns = ['LKOH', 'SBER', 'GAZP', 'NVTK', 'MOEX_0']
train_data = pd.merge(train_data, MOEX_1.shift(-1), left_index=True, right_index=True)
train_data.columns = ['LKOH', 'SBER', 'GAZP', 'NVTK', 'MOEX_0', 'MOEX_1']
train_data = pd.merge(train_data, MOEX_1.shift(-2), left_index=True, right_index=True)
train_data.columns = ['LKOH', 'SBER', 'GAZP', 'NVTK', 'MOEX_0', 'MOEX_1', 'MOEX_2']
train_data = train_data.apply(lambda x: np.log(x) - np.log(x).shift(1))
train_data.dropna(inplace=True)
train_data.head()

,LKOH,SBER,GAZP,NVTK,MOEX_0,MOEX_1,MOEX_2
1,0.000000,-0.000808,-0.002021,-0.001232,0.000185,0.000278,0.000370
2,0.001164,0.001212,-0.000947,0.001232,0.000278,0.000370,0.000093
3,-0.000485,0.002420,0.000000,-0.001540,0.000370,0.000093,-0.000278
4,0.000679,0.002214,-0.000474,0.000000,0.000093,-0.000278,-0.000278
5,-0.001067,-0.000281,-0.000733,0.000000,-0.000278,-0.000278,-0.000185


In [4]:
# Get MOEX daly return with testing data
MOEX_1 = DF(test_data.MOEX)
test_data = test_data.drop('MOEX', axis=1)
test_data = pd.merge(test_data, MOEX_1, left_index=True, right_index=True)
test_data.columns = ['LKOH', 'SBER', 'GAZP', 'NVTK', 'MOEX_0']
test_data = pd.merge(test_data, MOEX_1.shift(-1), left_index=True, right_index=True)
test_data.columns = ['LKOH', 'SBER', 'GAZP', 'NVTK', 'MOEX_0', 'MOEX_1']
test_data = pd.merge(test_data, MOEX_1.shift(-2), left_index=True, right_index=True)
test_data.columns = ['LKOH', 'SBER', 'GAZP', 'NVTK', 'MOEX_0', 'MOEX_1', 'MOEX_2']
test_data = test_data.apply(lambda x: np.log(x) - np.log(x).shift(1))
test_data.dropna(inplace=True)
test_data.head()

,LKOH,SBER,GAZP,NVTK,MOEX_0,MOEX_1,MOEX_2
1,-0.000485,0.00040,0.001168,0.003242,0.000093,0.000000,-0.000093
2,-0.000097,-0.00048,0.000994,-0.000925,0.000000,-0.000093,0.000278
3,0.000583,0.00008,0.000216,0.001233,-0.000093,0.000278,-0.000370
4,0.000582,-0.00080,0.000949,0.000000,0.000278,-0.000370,-0.000093
5,0.000097,-0.00056,-0.001079,-0.002005,-0.000370,-0.000093,0.000556


### We give the yield values to boolean variables
#### if the price rises, then 1 otherwise 0

In [5]:
for i in train_data.columns:
    train_data[i] = train_data[i].apply(lambda x: 1 if x > 0 else 0)
train_data.head()

,LKOH,SBER,GAZP,NVTK,MOEX_0,MOEX_1,MOEX_2
1,0,0,0,0,1,1,1
2,1,1,0,1,1,1,1
3,0,1,0,0,1,1,0
4,1,1,0,0,1,0,0
5,0,0,0,0,0,0,0


In [6]:
for i in test_data.columns:
    test_data[i] = test_data[i].apply(lambda x: 1 if x > 0 else 0)
test_data.head()

,LKOH,SBER,GAZP,NVTK,MOEX_0,MOEX_1,MOEX_2
1,0,1,1,1,1,0,0
2,0,0,1,0,0,0,1
3,1,1,1,1,0,1,0
4,1,0,1,0,1,0,0
5,1,0,0,0,0,0,1


### Plant decision trees

In [7]:
# data for training Decision Tree
x = train_data[['LKOH', 'SBER', 'GAZP', 'NVTK']]
yzero = train_data.MOEX_0
yone = train_data.MOEX_1
ytwo = train_data.MOEX_2

In [8]:
# get bets characteristic
dtc = DecisionTreeClassifier(random_state=0)
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(1, 4),
          'min_samples_leaf': range(1, 10)}
best_params_0 = GridSearchCV(dtc, params, cv=3).fit(x, yzero)
best_dtc_0 = best_params_0.best_estimator_
best_params_1 = GridSearchCV(dtc, params, cv=3).fit(x, yone)
best_dtc_1 = best_params_1.best_estimator_
best_params_2 = GridSearchCV(dtc, params, cv=3).fit(x, ytwo)
best_dtc_2 = best_params_2.best_estimator_

In [9]:
# data for training Decision Tree
x_t = test_data[['LKOH', 'SBER', 'GAZP', 'NVTK']]
yzero_t = test_data.MOEX_0
yone_t = test_data.MOEX_1
ytwo_t = test_data.MOEX_2

In [11]:
print('percentage of successful predictions: ', round(best_dtc_0.score(x_t, yzero_t) * 100, 2), '%', sep='')
print('percentage of successful predictions: ', round(best_dtc_0.score(x_t, yone_t)* 100, 2), '%', sep='')
print('percentage of successful predictions: ', round(best_dtc_0.score(x_t, ytwo_t)* 100, 2), '%', sep='')

percentage of successful predictions: 72.21%
percentage of successful predictions: 58.43%
percentage of successful predictions: 54.39%


## Summary
### As you can see, the success rate of decision tree predictions is 58% the next day and 54% the next day. What does it say about the random guessing of the result